## Imports

In [9]:
import fsspec
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy
import warnings
from IPython.display import HTML
from datetime import datetime
from dask_jobqueue import SLURMCluster
from dask.distributed import Client, progress
from dask import delayed, compute
from cartopy.io.shapereader import Reader
from cartopy.feature import ShapelyFeature
from matplotlib.animation import FuncAnimation

warnings.filterwarnings('ignore')

In [2]:
cluster = SLURMCluster(queue="seseml",
                       memory='50GB',
                       cores=40,
                       processes=1,
                       walltime='24:00:00', 
#                        scheduler_options={
#                            'host': '172.22.179.3:7224', 
#                            'dashboard_address': ':7798'
#                        }
                      )

cluster.scale(jobs=1)
client = Client(cluster)

In [3]:
%%bash
squeue -u $USER

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           1650239    seseml dask-wor alfonso8 PD       0:00      1 (None)
           1649851    seseml interact alfonso8  R    6:40:20      1 keeling-j01


In [4]:
def create_query(date, band='C01'):
    """
    Creates a query for listing GOES16 files stored in AWS bucket
    :param date: date to be queried. e.g datetime(2021, 10, 3, 12). Datetime python object
    :param band: Goes16 band to be queried e.g. C01
    :return: string with a
    """
    if data.hour != 0:
        prefix = f'ABI-L1b-RadF/{date:%Y}/{date.timetuple().tm_yday}/{date:%H}/OR_ABI-L1b-RadF-M6{band}_G16_s' \
                 f'{date:%Y}{date.timetuple().tm_yday}{date:%H}*'
    else:
        prefix = f'ABI-L1b-RadF/{date:%Y}/{date.timetuple().tm_yday}/{date:%H}/OR_ABI-L1b-RadF-M6{band}_G16_s' \
                 f'{date:%Y}{date.timetuple().tm_yday}*'
    return prefix


def calc_latlon(ds):
    # The math for this function was taken from
    # https://makersportal.com/blog/2018/11/25/goes-r-satellite-latitude-and-longitude-grid-projection-algorithm
    x = ds.x
    y = ds.y
    goes_imager_projection = ds.goes_imager_projection

    x, y = np.meshgrid(x, y)

    r_eq = goes_imager_projection.attrs["semi_major_axis"]
    r_pol = goes_imager_projection.attrs["semi_minor_axis"]
    l_0 = goes_imager_projection.attrs["longitude_of_projection_origin"] * (np.pi / 180)
    h_sat = goes_imager_projection.attrs["perspective_point_height"]
    H = r_eq + h_sat

    a = np.sin(x) ** 2 + (np.cos(x) ** 2 * (np.cos(y) ** 2 + (r_eq ** 2 / r_pol ** 2) * np.sin(y) ** 2))
    b = -2 * H * np.cos(x) * np.cos(y)
    c = H ** 2 - r_eq ** 2

    r_s = (-b - np.sqrt(b ** 2 - 4 * a * c)) / (2 * a)

    s_x = r_s * np.cos(x) * np.cos(y)
    s_y = -r_s * np.sin(x)
    s_z = r_s * np.cos(x) * np.sin(y)

    lat = np.arctan((r_eq ** 2 / r_pol ** 2) * (s_z / np.sqrt((H - s_x) ** 2 + s_y ** 2))) * (180 / np.pi)
    lon = (l_0 - np.arctan(s_y / (H - s_x))) * (180 / np.pi)
    ds = ds.assign_coords({"lat": np.nanmax(lat, axis=1),
                           "lon": np.nanmax(lon, axis=0)})
    ds.lat.attrs["units"] = "degrees_north"
    ds.lon.attrs["units"] = "degrees_east"
    return ds


def get_xy_from_latlon(ds, lats, lons):
    lat1, lat2 = lats
    lon1, lon2 = lons

    lat = ds.lat.data
    lon = ds.lon.data

    x = ds.x.data
    y = ds.y.data

    x, y = np.meshgrid(x, y)

    x = x[(lat >= lat1) & (lat <= lat2) & (lon >= lon1) & (lon <= lon2)]
    y = y[(lat >= lat1) & (lat <= lat2) & (lon >= lon1) & (lon <= lon2)]

    return ((np.nanmin(x), np.nanmax(x)), (np.nanmin(y), np.nanmax(y)))


def goes16_ds(date, band='C01'):
    str_bucket = 's3://noaa-goes16/'
    fs = fsspec.filesystem("s3", anon=True)
    query = create_query(date, band=band)
    files = [fs.open(f"s3://{i}") for i in sorted(fs.glob(f"{str_bucket}{query}"))]
    ds = xr.open_mfdataset(files, engine='h5netcdf', concat_dim='t', combine='nested')
    return calc_latlon(ds)


def normalization(da, vmin, vmax, gamma):
    norm = (da - vmin) / (vmax - vmin)
    norm = np.clip(norm, 0, 1)
    return np.power(norm, gamma)


def rad2tb(ds):
        """
        https://www.star.nesdis.noaa.gov/goesr/docs/ATBD/Imagery.pdf pag 22
        """
        fk1 = ds['planck_fk1']
        fk2 = ds['planck_fk2']
        bc1 = ds['planck_bc1']
        bc2 = ds['planck_bc2']
        bt = ((fk2 / (np.log((fk1 / ds.Rad) + 1))) - bc1) / bc2
        return bt


def vol_ash(ds11, ds13, ds14, ds15):
    r = rad2tb(ds15) - rad2tb(ds13)
    g = rad2tb(ds14) - rad2tb(ds11)
    b = rad2tb(ds13)
    # normalization
    r = normalization(r, vmin=-6.7, vmax=2.6, gamma=1)
    g = normalization(g, vmin=-6.0, vmax=6.3, gamma=1)
    b = normalization(b, vmin=243.6, vmax=302.4, gamma=1)
    rgb = np.dstack([r, g, b])
    return rgb


In [5]:
date = datetime(2023, 4, 10)

In [6]:
ds11 = goes16_ds(date, band="C11").sel(x=slice(-0.03, 0.03)).sel(y=slice(0.04, -0.01))
ds13 = goes16_ds(date, band="C13").sel(x=slice(-0.03, 0.03)).sel(y=slice(0.04, -0.01))
ds14 = goes16_ds(date, band="C14").sel(x=slice(-0.03, 0.03)).sel(y=slice(0.04, -0.01))
ds15 = goes16_ds(date, band="C15").sel(x=slice(-0.03, 0.03)).sel(y=slice(0.04, -0.01))

In [7]:
rgb = vol_ash(ds11=ds11.isel(t=0), ds13=ds13.isel(t=0), ds14=ds14.isel(t=0), ds15=ds15.isel(t=0))

In [11]:
dat = ds11.isel(t=0).metpy.parse_cf('Rad')
geos = dat.metpy.cartopy_crs
sat_h = ds11.goes_imager_projection.perspective_point_height
sat_h
x = ds11.x * sat_h
y = ds11.y * sat_h
cart_crs = ccrs.PlateCarree()
fig = plt.figure(dpi=150)
ax = fig.add_subplot(1, 1, 1, projection=geos)

im = ax.imshow(rgb, origin='upper',
                   transform=geos,
                   extent=(x.min(), x.max(), y.min(), y.max()))

gl = ax.gridlines(crs=cart_crs, draw_labels=True, 
                  linewidth=1, color="gray", alpha=0.3, linestyle="--")
plt.gca().xaxis.set_major_locator(plt.NullLocator())
ax.coastlines(resolution='10m', color='white', linewidth=0.5)
ax.add_feature(cartopy.feature.BORDERS, edgecolor='white', linewidth=0.5)
# shape_feature = ShapelyFeature(Reader('./shape/depto.shp').geometries(),
#                                 geos, facecolor='none')
# ax.add_feature(shape_feature)

def update_fig(t):
    rgb = rgb = vol_ash(ds11=ds11.isel(t=t), ds13=ds13.isel(t=t), ds14=ds14.isel(t=t), ds15=ds15.isel(t=t))
    im.set_array(rgb)
    

ani = FuncAnimation(fig, update_fig, frames=list(range(len(ds11.t))), interval=150)
plt.close()
HTML(ani.to_html5_video())